# 4.6 학습 및 검증 실시

- OpenPose 학습과 검증을 실시합니다, AWS의 GPU 머신으로 계산합니다.
- p2.xlarge로 45분 정도 걸립니다.


# 학습 목표

1.	OpenPose 학습을 구현할 수 있다

# 사전 준비

- 이전 장에서 구현한 클래스와 함수는 "utils" 폴더에 준비되어 있습니다


In [1]:
# 패키지 import
import random
import math
import time
import pandas as pd
import numpy as np
import torch
import torch.utils.data as data
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
# 초기 설정
# Setup seeds
torch.manual_seed(1234)
np.random.seed(1234)
random.seed(1234)

# DataLoader 작성

In [3]:
from utils.dataloader import make_datapath_list, DataTransform, COCOkeypointsDataset

# MS COCO 파일 경로 리스트 작성
train_img_list, train_mask_list, val_img_list, val_mask_list, train_meta_list, val_meta_list = make_datapath_list(
    rootpath="./data/")

# Dataset 작성
# 이 책에서는 데이터 양의 관계로 train을 val_list에서 작성하는 점에 주의
train_dataset = COCOkeypointsDataset(
    val_img_list, val_mask_list, val_meta_list, phase="train", transform=DataTransform())

# 이번에는 간이 학습으로써, 검증 데이터를 작성하지 않음
# val_dataset = CocokeypointsDataset(val_img_list, val_mask_list, val_meta_list, phase="val", transform=DataTransform())

# DataLoader 작성
batch_size = 32

train_dataloader = data.DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True)

# val_dataloader = data.DataLoader(
#    val_dataset, batch_size=batch_size, shuffle=False)

# 사전형 변수로 정리
# dataloaders_dict = {"train": train_dataloader, "val": val_dataloader}
dataloaders_dict = {"train": train_dataloader, "val": None}

# 네트워크 모델 작성

In [4]:
from utils.openpose_net import OpenPoseNet
net = OpenPoseNet()

# 손실함수를 정의

In [5]:
# 손실함수 설정
class OpenPoseLoss(nn.Module):
    """OpenPose의 손실함수 클래스"""

    def __init__(self):
        super(OpenPoseLoss, self).__init__()

    def forward(self, saved_for_loss, heatmap_target, heat_mask, paf_target, paf_mask):
        """
        손실함수 계산.

        Parameters
        ----------
        saved_for_loss : OpenPoseNet의 출력(리스트)

        heatmap_target : [num_batch, 19, 46, 46]
            정답 부위의 어노테이션 정보

        heatmap_mask : [num_batch, 19, 46, 46]
            heatmap 화상의 mask

        paf_target : [num_batch, 38, 46, 46]
            정답 PAF의 어노테이션 정보

        paf_mask : [num_batch, 38, 46, 46]
            PAF 화상의 mask

        Returns
        -------
        loss : 텐서
            손실값
        """

        total_loss = 0
        # 스테이지마다 계산합니다
        for j in range(6):

            # PAFs 및 heatmaps에서 마스크된 부분(paf_mask=0 등)은 무시
            # PAFs
            pred1 = saved_for_loss[2 * j] * paf_mask
            gt1 = paf_target.float() * paf_mask

            # heatmaps
            pred2 = saved_for_loss[2 * j + 1] * heat_mask
            gt2 = heatmap_target.float()*heat_mask

            total_loss += F.mse_loss(pred1, gt1, reduction='mean') + \
                F.mse_loss(pred2, gt2, reduction='mean')

        return total_loss

criterion = OpenPoseLoss()

# 최적화 기법 설정

In [6]:
optimizer = optim.SGD(net.parameters(), lr=1e-2,
                      momentum=0.9,
                      weight_decay=0.0001)

# 학습 실시

In [7]:
# 모델을 학습시키는 함수 작성
def train_model(net, dataloaders_dict, criterion, optimizer, num_epochs):

    # GPU가 사용 가능한지 확인
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print("사용 장치: ", device)

    # 네트워크를 GPU로
    net.to(device)

    # 네트워크가 어느 정도 고정되면, 고속화시킨다
    torch.backends.cudnn.benchmark = True

    # 화상의 매수
    num_train_imgs = len(dataloaders_dict["train"].dataset)
    batch_size = dataloaders_dict["train"].batch_size

    # 반복 카운터 설정
    iteration = 1

    # epoch 루프
    for epoch in range(num_epochs):

        # 개시 시간을 저장
        t_epoch_start = time.time()
        t_iter_start = time.time()
        epoch_train_loss = 0.0  # epoch의 손실합
        epoch_val_loss = 0.0  # epoch의 손실합

        print('-------------')
        print('Epoch {}/{}'.format(epoch+1, num_epochs))
        print('-------------')

        # epoch별 훈련 및 검증 루프
        for phase in ['train', 'val']:
            if phase == 'train':
                net.train()  # 모델을 훈련 모드로
                optimizer.zero_grad()
                print('(train)')

            # 이번에는 검증을 생략
            else:
                continue
                # net.eval()   # 모델을 검증 모드로
                # print('-------------')
                # print('(val)')

            # 데이터 로더에서 minibatch씩 꺼내는 루프
            for imges, heatmap_target, heat_mask, paf_target, paf_mask in dataloaders_dict[phase]:
                # 미니 배치 크기가 1이면, 배치 노멀라이제이션에서 에러가 발생하므로 피한다
                if imges.size()[0] == 1:
                    continue

                # GPU가 사용 가능하면 GPU로 데이터를 보낸다
                imges = imges.to(device)
                heatmap_target = heatmap_target.to(device)
                heat_mask = heat_mask.to(device)
                paf_target = paf_target.to(device)
                paf_mask = paf_mask.to(device)

                # optimizer 초기화
                optimizer.zero_grad()

                # 순전파(forward) 계산
                with torch.set_grad_enabled(phase == 'train'):
                    # (out6_1, out6_2)는 사용하지 않으므로 _ 로 대체
                    _, saved_for_loss = net(imges)

                    loss = criterion(saved_for_loss, heatmap_target,
                                     heat_mask, paf_target, paf_mask)
                    del saved_for_loss
                    # 훈련시에는 역전파
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                        if (iteration % 10 == 0):  # 10iter에 1번, loss를 표시
                            t_iter_finish = time.time()
                            duration = t_iter_finish - t_iter_start
                            print('반복 {} || Loss: {:.4f} || 10iter: {:.4f} sec.'.format(
                                iteration, loss.item()/batch_size, duration))
                            t_iter_start = time.time()

                        epoch_train_loss += loss.item()
                        iteration += 1

                    # 검증시
                    # else:
                        #epoch_val_loss += loss.item()

        # epoch의 phase별 loss와 정답률
        t_epoch_finish = time.time()
        print('-------------')
        print('epoch {} || Epoch_TRAIN_Loss:{:.4f} ||Epoch_VAL_Loss:{:.4f}'.format(
            epoch+1, epoch_train_loss/num_train_imgs, 0))
        print('timer:  {:.4f} sec.'.format(t_epoch_finish - t_epoch_start))
        t_epoch_start = time.time()

    # 마지막 네트워크를 저장한다
    torch.save(net.state_dict(), 'weights/openpose_net_' +
               str(epoch+1) + '.pth')


In [8]:
# 학습 및 검증을 실행한다
num_epochs = 2
train_model(net, dataloaders_dict, criterion, optimizer, num_epochs=num_epochs)

사용 장치:  cuda:0
-------------
Epoch 1/2
-------------
(train)
반복 10 || Loss: 0.0094 || 10iter: 113.7127 sec.
반복 20 || Loss: 0.0082 || 10iter: 90.4145 sec.
반복 30 || Loss: 0.0069 || 10iter: 88.4890 sec.
반복 40 || Loss: 0.0058 || 10iter: 90.9961 sec.
반복 50 || Loss: 0.0050 || 10iter: 90.8274 sec.
반복 60 || Loss: 0.0042 || 10iter: 89.7553 sec.
반복 70 || Loss: 0.0038 || 10iter: 91.1155 sec.
반복 80 || Loss: 0.0031 || 10iter: 91.3307 sec.
반복 90 || Loss: 0.0027 || 10iter: 91.7214 sec.
반복 100 || Loss: 0.0026 || 10iter: 92.2645 sec.
반복 110 || Loss: 0.0023 || 10iter: 91.7421 sec.
반복 120 || Loss: 0.0020 || 10iter: 90.7930 sec.
반복 130 || Loss: 0.0020 || 10iter: 91.3045 sec.
반복 140 || Loss: 0.0019 || 10iter: 91.6105 sec.
반복 150 || Loss: 0.0016 || 10iter: 90.2619 sec.
-------------
epoch 1 || Epoch_TRAIN_Loss:0.0043 ||Epoch_VAL_Loss:0.0000
timer:  1462.0789 sec.
-------------
Epoch 2/2
-------------
(train)
반복 160 || Loss: 0.0017 || 10iter: 64.3399 sec.
반복 170 || Loss: 0.0017 || 10iter: 91.2324 sec.
반복 180

끝